In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from utils import (
    process_complaints_batch,
    analyze_model_similarity,
    run_pipeline_deduplicate,
    export_results,
    construction
)

load_dotenv()
api_key = os.getenv("OPENROUTER_API_KEY")
models = [
    "openai/gpt-4o",
    "openai/gpt-4o-mini",
    "openai/gpt-4.1",
    "openai/gpt-4.1-mini",
    "openai/gpt-5-mini",
    "anthropic/claude-3.5-sonnet",
    "anthropic/claude-3.7-sonnet",
    "anthropic/claude-sonnet-4",
    "meta-llama/llama-3.3-70b-instruct",
    "meta-llama/llama-4-maverick",
    "google/gemini-2.5-flash",
    "google/gemini-2.5-flash-lite",
    "deepseek/deepseek-chat-v3-0324",
    "x-ai/grok-4",
    "x-ai/grok-3-mini"
]


## Data Preprocessing
Length of Allegations:Only keep rows where the Allegations text contains at least 200 words.

Date Filtering: keep rows where either date of Date_Complaint_Received,Date_Notice is on or after January 1, 2000.


In [2]:
notebook_path = os.getcwd()
file_path = os.path.join(notebook_path, "data", "raw_disclosure_complaints_panel.parquet")
df = pd.read_parquet(file_path)
df.insert(0, "Index", range(1, len(df) + 1))
print(f"OriginalNumber of rows: {len(df)}")

# Keep rows where Allegations has at least 200 words
df = df[df["Allegations"].str.split().str.len() >= 200].copy()

#Date Filtering: keep rows where either date of Date_Complaint_Received,Date_Notice is on or after January 1, 2000.
date_cols = ["Date_Complaint_Received", "Date_Notice"]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")

cutoff = pd.Timestamp("2000-01-01")

mask = df[date_cols].ge(cutoff).any(axis=1)
df = df[mask].copy()
df = df.sort_values("Date_Complaint_Received", ascending=False).drop_duplicates(subset="Allegations", keep="first")
df = df[df["Allegations"].str.contains("unsuit", case=False, na=False)].copy()
print(f"Number of records after filtering: {len(df)}")

# Select a subset for testing
test=10
df_test = df.iloc[0:test].copy()


OriginalNumber of rows: 160349
Number of records after filtering: 121


In [3]:
df_test

,Index,X1,CRD#,Name,Disclosure_Type,Current_Status,Disclosure_Number,Reporting_Source,Employing_Firm,Allegations,...,Individual_Contribution_Amount,Arbitration_Claim,Date_Notice,Arbitration_Pending?,Disposition,Disposition_Date,Disposition_Detail,Monetary_Compensation_Amount,Statement,Date_Complaint_Received2
99109,99110,10,5752036,CHARLES OKAI BOTCHWAY,Customer Dispute,Pending,1,Broker,"MSC-BD, LLC",[REDACTED] and the [REDACTED] Trust (together ...,...,None,None,2019-04-01,Yes,None,None,None,None,Broker Statement The Registered Representative...,2019-04-01
103431,103432,399,205552,ROBERT MALONE FEHRMAN,Customer Dispute,Pending,1,Broker,"MSC-BD, LLC",[REDACTED] and the [REDACTED] Trust (together ...,...,None,None,2019-04-01,Yes,None,None,None,None,Broker Statement The Registered Representative...,2019-04-01
105462,105463,23,6065805,RYAN OWEN TARJANYI,Customer Dispute,Settled,1,Broker,"Bankers Life Securities, Inc.","In a written complaint, dated October 19, 2018...",...,$0.00,None,NaT,None,None,None,None,None,Broker Statement I was not a party in this set...,2019-02-19
35305,35306,999,723366,GARY CARL MASTRODONATO,Customer Dispute,Pending,1,Broker,"Resource Horizons Group, LLC, Madison Avenue S...",Unsuitable investments; negligence; breach of ...,...,None,None,2019-03-11,Yes,None,None,None,None,Alleged Damages Amount Statement of Claim requ...,2018-07-30
107609,107610,10,6232318,DANIEL THOMAS FITZPATRICK,Customer Dispute,Pending,1,Firm,Chrysalis Capital Group LLC,"August 2014. [REDACTED], a man in his late 70'...",...,None,None,NaT,None,None,None,None,None,None,2018-02-16
85187,85188,42,5050708,JAVIER ANTONIO LOPEZ,Customer Dispute,Pending,4,Broker,UBS FINANCIAL SERVICES INC.,©2019 FINRA. All rights reserved. Report about...,...,None,None,NaT,None,None,None,None,None,None,2017-08-07
74694,74695,195,4685974,FEDERICO ANDRES LOPEZ JUARBE,Customer Dispute,Pending,8,Broker,UBS FINANCIAL SERVICES INC.,Time frame: 2013-2016 The claimants in this ca...,...,None,None,NaT,None,None,None,None,None,None,2017-08-07
118644,118645,12,1485103,WILLIAM FRANCIS THON,Customer Dispute,Settled,1,Firm,"UBS Financial Services, Inc",The Customers allege unsuitable investment rec...,...,None,None,NaT,None,None,None,None,None,None,2017-08-04
79861,79862,34,4865317,MARIA JOSE MIRANDA CRUZ MS.,Customer Dispute,Settled,1,Broker,"Popular Securities, LLC",Claimants allege that Financial Consultant inv...,...,None,None,NaT,None,None,None,None,None,Broker Statement THE FINANCIAL CONSULTANT DENI...,2017-05-30
87928,87929,617,52604,STEPHEN NELSON CONRAD,Customer Dispute,Settled,1,Broker,"WINSLOW, EVANS & CROCKER, INC.","ON NOVEMBER 6, 2016, CUSTOMER SENT AN E-MAIL T...",...,"$10,176.73",None,NaT,None,None,None,None,None,"Broker Statement ON NOVEMBER 6, 2016, CUSTOMER...",2016-11-06


# Stage 1: Extract
### Stage 1.1 Using subset to find the better model

In [ ]:
num_tests=2
subsetcompare, _ = await process_complaints_batch(df, test=10, models=models,output_file="stage2_1modelchosen.json",n=num_tests)

Processing 10 complaints with 15 models...
Generated 20 prompts for 15 models
Batch completed


Processing responses: 100%|██████████| 300/300 [00:00<00:00, 46076.06it/s]

JSON parse error for index 99110: JSON parsing failed after cleaning: Expecting value: line 1 column 1 (char 0)
Input: Based on the complaint case, I will extract a list of evaluation criteria that focus on suitability concerns in financial advice and investment compliance scenarios. Here is the output in valid JSON:


{
  "evaluation_criteria": [
    {
      "question": "Did the advisor assess the client's risk tolerance and investment goals before recommending high-risk, illiquid investments?",
      "explanation": "The case involved Claimants who were allegedly recommended high-risk, illiquid investments, including BDCs and LPs, without consideration for their risk tolerance or investment goals."
    },
    {
      "question": "Were the recommended investments suitable for the client's age, financial situation, and investment experience?",
      "explanation": "The Claimants were elderly and allegedly had their retirement savings invested in high-risk, illiquid investments, raising 

We evaluate how consistently each model performs when analyzing a subset of cases from the full dataset:
For each model, we extract the evaluation questions and measure two key aspects: the number of questions generated and the internal similarity among those questions. 
To select a representative model, we apply a two-step criterion. First, we restrict attention to models whose number of questions corresponds to the mode (the most frequently observed question count across all models). Second, within this modal group, we identify the model with the lowest similarity score, as this reflects the weakest ability to produce diverse and substantive evaluation criteria.

### Stage1.2 Model choosing to extract

In [5]:
df_summary, df_casewise, chosen_model = analyze_model_similarity(subsetcompare)
chosen_model
print("\n=== Analysis Results ===")
print("\n=== Model Summary ===")
df_summary

Analyzed 290 cases
Models: ['openai/gpt-4o' 'openai/gpt-4o-mini' 'openai/gpt-4.1'
 'openai/gpt-4.1-mini' 'openai/gpt-5-mini' 'anthropic/claude-3.5-sonnet'
 'anthropic/claude-3.7-sonnet' 'anthropic/claude-sonnet-4'
 'meta-llama/llama-3.3-70b-instruct' 'google/gemini-2.5-flash'
 'google/gemini-2.5-flash-lite' 'deepseek/deepseek-chat-v3-0324'
 'x-ai/grok-4' 'x-ai/grok-3-mini' 'meta-llama/llama-4-maverick']
Mean Avg_Num_Questions: 5.08
Standard Deviation: 1.02

=== Model Selection Process ===
Models considered: ['anthropic/claude-3.5-sonnet', 'anthropic/claude-3.7-sonnet', 'anthropic/claude-sonnet-4', 'deepseek/deepseek-chat-v3-0324', 'google/gemini-2.5-flash', 'google/gemini-2.5-flash-lite', 'meta-llama/llama-3.3-70b-instruct', 'openai/gpt-4.1', 'openai/gpt-4.1-mini', 'openai/gpt-4o', 'openai/gpt-4o-mini', 'openai/gpt-5-mini', 'x-ai/grok-3-mini', 'x-ai/grok-4']
Model chosen: anthropic/claude-3.5-sonnet
  - Avg Questions: 4.85)
  - Similarity Score: 0.531 (lower = more diverse)
  - Cases P

,Model,Avg_Num_Questions,Std_Num_Questions,Case_Count,Avg_SimilarityScore,Std_SimilarityScore
0,anthropic/claude-3.5-sonnet,4.85,0.366,20,0.531,0.118
1,anthropic/claude-3.7-sonnet,5.45,0.999,20,0.641,0.063
2,anthropic/claude-sonnet-4,4.75,0.550,20,0.656,0.054
3,deepseek/deepseek-chat-v3-0324,5.50,0.688,20,0.707,0.040
4,google/gemini-2.5-flash,4.30,1.218,20,0.717,0.059
5,google/gemini-2.5-flash-lite,5.40,1.314,20,0.677,0.073
6,meta-llama/llama-3.3-70b-instruct,5.00,0.562,20,0.700,0.060
7,meta-llama/llama-4-maverick,5.10,0.738,10,0.692,0.062
8,openai/gpt-4.1,5.00,0.725,20,0.706,0.051
9,openai/gpt-4.1-mini,4.65,1.089,20,0.747,0.054


In [ ]:
df_summary.to_csv("pipeline_output/stage1_2modelselection.csv", index=False)

# Print the case detail results
# print("\n=== Case Details  ===")
# df_casewise[['Model', 'Complaint_Index', 'Num_Questions', 'SimilarityScore']]

### Stage 1.3 Extract the rest complaint using the chosen model

In [ ]:
# Extract existing results for chosen model
half_len = len(subsetcompare) // num_tests
subset_half = subsetcompare[:half_len]
resultsfull = [item for item in subset_half if item.get("Model") == chosen_model]

# Process remaining complaints using the same batch function
remaining_results, _ = await process_complaints_batch(
    df=df.iloc[test:],  # Remaining data
    test=len(df) - test,  # Number of remaining records
    models=[chosen_model],  # Only use chosen model
    output_file=f"stage1_3ExtractQuestions_{chosen_model.replace('/', '_')}.json",
)

# Combine results
resultsfull.extend(remaining_results)
print(f"Total results: {len(resultsfull)} (original: {len([item for item in subsetcompare if item.get('Model') == chosen_model])}, new: {len(remaining_results)})")


Processing 111 complaints with 1 models...
Generated 111 prompts for 1 models
Batch completed


Processing responses: 100%|██████████| 111/111 [00:00<00:00, 55497.41it/s]


=== Processing Complete ===
Total results: 111, Successful: 111, Errors: 0
Results saved to: stage2_3ExtractQuestions_anthropic_claude-3.5-sonnet.json
Total results: 121 (original: 20, new: 111)


In [16]:
chosen_model

'anthropic/claude-3.5-sonnet'

# Stage2 Deduplication

In [ ]:
# file = f"output/stage2_3ExtractQuestions_openai_gpt-4.1.json"
# with open(file, "r", encoding="utf-8") as f:
#     resultsfull = json.load(f)

In [18]:
dedup_model = "openai/gpt-4.1"

items, final_comparison, question_list = await run_pipeline_deduplicate(
        resultsfull, dedup_model, max_iterations=10, max_concurrent=10
    )

[Iter 1] items: 566 -> 409 | groups: 121 -> 25 | queued_fallback: 0
[Iter 2] items: 409 -> 290 | groups: 25 -> 5 | queued_fallback: 3
[Iter 3] items: 298 -> 176 | groups: 8 -> 2 | queued_fallback: 2
[Iter 4] items: 188 -> 35 | groups: 4 -> 1 | queued_fallback: 1
[Iter 5] items: 77 -> 53 | groups: 2 -> 1 | queued_fallback: 0


In [23]:
export_results(items, final_comparison, dedup_model, resultsfull)


✅ Export completed:
📁 Main folder: stage3_duplicate
📁 Model folder: output\stage3_duplicate\openai_gpt-4.1
💾 Complete data (JSON): output\stage3_duplicate\openai_gpt-4.1\Stage3_complete_results_20251008_154201.json
📋 Final comparison (MD): output\stage3_duplicate\openai_gpt-4.1\final_comparison_20251008_154201.md
🎯 Last round questions (MD): output\stage3_duplicate\openai_gpt-4.1\last_round_questions_20251008_154201.md


In [ ]:
from utils.complaint_Stage3choiceClass import classify_questions
from collections import Counter

classified = await classify_questions(question_list,
    dedup_model,
    max_concurrent = 10,
    classtype="temporal"
)

Saved question_list to output/stage3_classified_openai_gpt-4.1_20251008_155052.json


In [ ]:
cat_counter = Counter(it.get("category", "Unknown") for it in classified)
title_counter = Counter(it.get("title", "Unknown") for it in classified)

total = len(question_list)
pre_n = cat_counter.get("pre-sale", 0)
post_n = cat_counter.get("post-sale", 0)
unk_n = cat_counter.get("Unknown", 0)

print(
        f"[classify_questions] Total: {total} | pre-sale: {pre_n} | post-sale: {post_n} | Unknown: {unk_n}"
    )


[classify_questions] Total: 53 | pre-sale: 39 | post-sale: 14 | Unknown: 0


In [ ]:
# print all titles under "pre-sale"
pre_sale_items = [it for it in classified if it.get("category") == "pre-sale"]

print("\n[Pre-sale topics]")
for it in pre_sale_items:
    print(f"- {it.get('title', 'Unknown')} | Q: {it.get('question', it.get('text', ''))}")


[Pre-sale topics]
- Product Explanation and Documentation | Q: Did the advisor clearly explain and document product features, benefits, risks, costs, limitations, and how they align with the client's financial needs and objectives?
- Supervisory Controls for Detecting Unsuitable Activities | Q: Did the firm or advisor have adequate supervisory controls and systems to detect, monitor, and prevent unauthorized or unsuitable activities, including trading holds, delays, and recommendation patterns?
- Advisor Pressure and Decision Time | Q: Did the advisor allow clients sufficient time to make informed decisions without applying pressure tactics or artificial urgency?
- Advisor Warnings and Documentation for Aggressive Positions | Q: Did the advisor provide clear written warnings and maintain documentation when clients chose to maintain aggressive positions against recommendations?
- Advisor Response to Negative Developments in Recommendations | Q: Did the advisor adequately consider and r

# Stage3 Rubric Generation

In [42]:
final_table = await construction(pre_sale_items,dedup_model)

Updated 39 items with options
Saved question_list to output/stage4_classified_20251008_163836.json


In [43]:
for i, item in enumerate(final_table, 1):
    print(f"\nQuestion {i}: {item['question']}\n")
    for opt, desc in item["options"].items():
        print(f"  {opt}. {desc}")


Question 1: Did the advisor clearly explain and document product features, benefits, risks, costs, limitations, and how they align with the client's financial needs and objectives?

  A. The advisor provides a comprehensive, clear, and client-specific explanation of all relevant product features, benefits, risks, costs, and limitations. Documentation is thorough and tailored, explicitly linking each aspect of the product to the client’s stated financial needs and objectives. The advisor ensures client understanding through active engagement (e.g., asking clarifying questions, providing written summaries), and all disclosures are properly recorded in the client file.
  B. The advisor explains and documents most product features, benefits, risks, costs, and limitations, with only minor omissions or generic language. The alignment with the client’s financial needs and objectives is generally clear, though some connections may lack detail. Client understanding is checked, but not as thoro